# Can you sum some cards?
## [Fiddler, Feb. 16, 2024](https://thefiddler.substack.com/p/can-you-sum-some-cards)

### solution by [Laurent Lessard](https://laurentlessard.com)

This heavily optimized Julia code finds for $N=1,2,\dots,7$ the largest $k$ such that there exists a sequence $a_1,\dots,a_k \in \{1,2,\dots,N\}$ such that for all values $p,q,r$ with $1\leq p\leq q\lt r\leq k$ it is never true that
$$\sum_{m=p}^{q}{a_m} = \sum_{n=q+1}^{r}{a_n}.$$
I stopped at $N=7$ because in spite of all the optimization, this is a fundamentally hard combinatorial problem and solving $N=8$ would likely take all day!

In [1]:
function sorted_intersect(x::Vector{Int64}, y::Vector{Int64})
    n = length(x)
    i, j = 1, 1
    while i <= n && j <= n
        if x[i] < y[j]
            i += 1
        elseif y[j] < x[i]
            j += 1
        else
            return true
        end
    end
    return false
end

function slap(x::Vector{Int64})   
    v = cumsum(reverse(x))
    return sorted_intersect(v, 2*v)
end

function extend_sequences(sequences, N::Int64)
    new_sequences = []
    for seq in sequences
        for i in 1:N
            if !slap([seq; i])  # Assuming this checks validity efficiently
                push!(new_sequences, [seq; i])
            end
        end
    end
    return new_sequences
end

function longest_no_slap_sequences(N::Int64)
    sequences = [[i] for i in 1:N]
    while true
        new_sequences = extend_sequences(sequences, N)
        if isempty(new_sequences)
            break
        else
            sequences = new_sequences
        end
    end
    return sequences[1]
end

longest_no_slap_sequences (generic function with 1 method)

In [4]:
exemplars = []
times = []
for N = 1:7
    t = @elapsed begin
        e = longest_no_slap_sequences(N)
    end
    push!(exemplars, e)
    push!(times, t)
    println(t,",  ", length(e))
end

5.7e-6,  1
9.3e-6,  3
4.08e-5,  7
0.0004714,  9
0.0103454,  15
1.1435513,  21
49.0357215,  31


In [5]:
for (t,e) in zip(times, exemplars)
    println( "time: $t seconds, length $(length(e)), example: $e" )
end

time: 5.7e-6 seconds, length 1, example: [1]
time: 9.3e-6 seconds, length 3, example: [1, 2, 1]
time: 4.08e-5 seconds, length 7, example: [1, 3, 2, 3, 1, 3, 2]
time: 0.0004714 seconds, length 9, example: [1, 4, 3, 4, 2, 4, 3, 4, 1]
time: 0.0103454 seconds, length 15, example: [1, 3, 2, 4, 5, 3, 4, 5, 1, 3, 2, 4, 5, 3, 4]
time: 1.1435513 seconds, length 21, example: [3, 5, 4, 6, 4, 3, 5, 4, 6, 5, 6, 2, 6, 5, 6, 1, 3, 5, 6, 1, 6]
time: 49.0357215 seconds, length 31, example: [1, 7, 6, 7, 5, 7, 6, 4, 3, 4, 6, 7, 5, 7, 6, 7, 1, 7, 6, 7, 5, 7, 6, 4, 3, 4, 6, 7, 5, 7, 6]
